In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import zipfile
%matplotlib inline

## Canonicalization

In [2]:
with open('county_and_state.csv') as f:
    county_and_state = pd.read_csv(f)
    
with open('county_and_population.csv') as f:
    county_and_pop = pd.read_csv(f)    

Suppose we'd like to join these two tables. Unfortunately, we can't, because the strings representing the county names don't match, as seen below.

In [3]:
county_and_state

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


In [4]:
county_and_pop

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


 Before we can join them, we'll do what I call **canonicalization**.

Canonicalization: A process for converting data that has more than one possible representation into a "standard", "normal", or canonical form (definition via Wikipedia).

In [5]:
def canonicalize_county(county_name):
    return (
        county_name
        .lower()               # lower case
        .replace(' ', '')      # remove spaces
        .replace('&', 'and')   # replace &
        .replace('.', '')      # remove dot
        .replace('county', '') # remove county
        .replace('parish', '') # remove parish
    )

In [6]:
county_and_pop['clean_county'] = county_and_pop["County"].map(canonicalize_county)
county_and_state['clean_county'] = county_and_state["County"].map(canonicalize_county)

In [7]:
county_and_pop


,County,Population,clean_county
0,DeWitt,16798,dewitt
1,Lac Qui Parle,8067,lacquiparle
2,Lewis & Clark,55716,lewisandclark
3,St. John the Baptist,43044,stjohnthebaptist


In [8]:
county_and_pop.merge(county_and_state, left_on= 'clean_county',right_on='clean_county')

,County_x,Population,clean_county,County_y,State
0,DeWitt,16798,dewitt,De Witt County,IL
1,Lac Qui Parle,8067,lacquiparle,Lac qui Parle County,MN
2,Lewis & Clark,55716,lewisandclark,Lewis and Clark County,MT
3,St. John the Baptist,43044,stjohnthebaptist,St John the Baptist Parish,LS


## Processing Data from a Text Log Using Basic Python

In [20]:
with open('log.txt', 'r') as f:
    log_lines = f.readlines()

In [23]:
log_lines[0][20:31]

'26/Jan/2014'

In [22]:
log_lines[0].split('[')

['169.237.46.168 - - ',
 '26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n']

Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. That is, slicing by some fixed offset isn't going to work.

Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [12]:
first = log_lines[0]
first

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

In [24]:
pertinent = first.split("[")[1].split(']')[0]
day, month, rest = pertinent.split('/')
year, hour, minute, rest = rest.split(':')
seconds, time_zone = rest.split(' ')
day, month, year, hour, minute, seconds, time_zone

('26', 'Jan', '2014', '10', '47', '58', '-0800')

A much more sophisticated but common approach is to extract the information we need using a regular expression.

In [25]:
import re
pattern = r'\[(\d+)/(\w+)/(\d+):(\d+):(\d+):(\d+) (.+)\]'
day, month, year, hour, minute, seconds, time_zone = re.search(pattern,first).groups()
day, month, year, hour, minute, seconds, time_zone

('26', 'Jan', '2014', '10', '47', '58', '-0800')

Or alternately using the `findall` method:

In [26]:
pattern = r'\[(\d+)/(\w+)/(\d+):(\d+):(\d+):(\d+) (.+)\]'
day, month, year, hour, minute, seconds, time_zone = re.findall(pattern,first)[0]
day, month, year, hour, minute, seconds, time_zone

('26', 'Jan', '2014', '10', '47', '58', '-0800')

Note: We can return the results as a Series:

In [27]:
cols = ["Day", "Month", "Year", "Hour", "Minute", "Second", "Time Zone"]
def log_entry_to_series(line):
    return pd.Series(re.search(pattern, line).groups(), index = cols)

log_entry_to_series(first)

Day             26
Month          Jan
Year          2014
Hour            10
Minute          47
Second          58
Time Zone    -0800
dtype: object

In [28]:
pattern = r"\[(\d{2})/(\w{3})/(\d{4})"
matches = re.findall(pattern, first)
day, month, year = matches[0]
print(day,month,year)

26 Jan 2014


In [35]:
text = "My Phone number is 145-6576-4295";
pattern = r"1[0-9]{2}-[0-9]{4}-[0-9]{4}"
m = re.findall(pattern, text)
print(m)
print(type(m[0]))

['145-6576-4295']
<class 'str'>


In [34]:
text = "My Phone number is 145-6576-4295";
pattern = r"(1[0-9]{2})-([0-9]{4})-([0-9]{4})"
m = re.findall(pattern, text)
print(m)
print(type(m[0]))

[('145', '6576', '4295')]
<class 'tuple'>


And using this function we can create a DataFrame of all the time information.

In [31]:
text = '<div><td valign="top">Moo</td></div>'
pattern = r"<[^>]+>" 
m = re.findall(pattern, text)
cleaned = re.sub(pattern, '', text)
print(cleaned)

['<div>', '<td valign="top">', '</td>', '</div>']

Moo
